In [49]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time
from datetime import date
import glob


np.random.seed(1337)

today = str(date.today())
end_date = today


path = r'./data/final'  # use your path
allFiles = glob.glob(path + "/2*.csv")

dfTrain = pd.DataFrame()

for _file in allFiles[:-5]:
        try:
            t = pd.read_csv(_file, header=0)
            if "LastGame" not in t:
                print("LastGame not in " + _file)
            k = len(t[((t["Final"]==0) & (t["injured"]==0))])
            #if k > 0:
            #print("Found something in: " + _file + " count: " + str(k))
            dfTrain = pd.concat([dfTrain,t])
        except:
            print("There was an exception with " + _file)
            pass

print("Last file loaded: " + str(len(allFiles)))
test_files = allFiles[-5:-1]
dfTest = pd.DataFrame()
for _file in test_files:
    print("Test file " + _file)
    dfTest = pd.concat([dfTest, pd.read_csv(_file,header=0)])


predict_file = './data/final/2017-02-01.csv'
print("Predict file " + predict_file)
dfPredict = pd.read_csv(predict_file, header=0)

out_file = predict_file.replace("final","final2")
print("Out file " + out_file)

Last file loaded: 41
Test file ./data/final/2017-01-28.csv
Test file ./data/final/2017-01-29.csv
Test file ./data/final/2017-01-30.csv
Test file ./data/final/2017-01-31.csv
Predict file ./data/final/2017-02-01.csv
Out file ./data/final2/2017-02-01.csv


In [50]:
print(len(dfTrain))
dfTrain.loc[:,('LastGame','fvalue','Final')].fillna(0.0, inplace=True)
dfTrain.loc[(dfTrain['Final'] == 0),"Final"] = dfTrain[(dfTrain['Final'])==0]['4GameAvg']
dfTrain = dfTrain.dropna()
#dfTrain[(dfTrain["Final"]==0)] = dfTrain[(dfTrain["Final"]==0)][["4GameAvg"]]
print(len(dfTrain))

print(len(dfTest))
dfTest.loc[:,('LastGame','fvalue','Final')].fillna(0.0, inplace=True)
dfTest.loc[(dfTest['Final'] == 0),"Final"] = dfTest[(dfTest['Final'])==0]['4GameAvg']
dfTest = dfTest.dropna()
print(len(dfTest))

7759
7649
716
708


In [51]:
dfTrain.loc[:,('4GameAvg','LastGame','fvalue','Final')].head(10)

,4GameAvg,LastGame,fvalue,Final
0,55.812,81.25,0.183,73.250
1,52.938,43.75,0.180,42.500
2,48.750,69.50,0.228,55.250
3,41.250,26.75,0.255,36.250
4,37.000,44.75,0.122,41.000
5,32.188,32.75,0.256,16.000
6,41.562,36.25,0.271,50.250
7,38.750,21.25,0.293,40.750
8,40.188,43.25,0.288,40.188
9,39.562,44.75,0.244,49.500


In [52]:
dfTest.columns[pd.isnull(dfTest).any()]

Index([], dtype='object')

In [53]:
print(len(dfPredict))
dfPredict = dfPredict.fillna(0)
print(len(dfPredict))

358
358


In [54]:
scaler = StandardScaler()
features = ['Salary','AvgPointsPerGame','7GameAvg','FloorAvg'
            ,'4GameAvg','value','fvalue','atHome','LastGame','O/U','odds',"dvp"]

In [55]:
df_train = dfTrain
X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train["Final"].values

In [56]:
df_test = dfTest
X_test = scaler.transform(df_test[features].values)
y_test = df_test["Final"].values

In [57]:
df_train.describe()

,Salary,AvgPointsPerGame,atHome,injured,7GameAvg,FloorAvg,4GameAvg,LastGame,dvp,value,fvalue,odds,O/U,penalty,Final
count,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000,7649.000000
mean,4245.195450,17.578403,0.500588,0.074520,17.721307,11.815087,17.516462,17.091483,15.366976,25.471173,0.200132,0.024121,208.097072,-929.776410,17.738466
std,1748.606959,11.738052,0.500032,0.262632,12.336560,10.052983,12.739055,14.677000,8.386971,10.491642,0.138597,7.011350,27.144365,2737.851237,14.169845
min,3000.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,18.000000,0.000000,-18.000000,1.000000,-9158.347000,-1.000000
25%,3000.000000,8.939000,0.000000,0.000000,8.357000,3.925000,7.375000,5.000000,8.500000,18.000000,0.082000,-5.000000,205.000000,-8.815000,5.875000
50%,3400.000000,15.936000,1.000000,0.000000,15.786000,9.714000,15.250000,14.000000,15.000000,20.400000,0.199000,1.000000,211.500000,-1.435000,14.750000
75%,5000.000000,24.206000,1.000000,0.000000,24.821000,16.797000,24.875000,25.750000,22.500000,30.000000,0.295000,5.000000,217.000000,3.632000,26.000000
max,13000.000000,65.219000,1.000000,1.000000,70.321000,54.533000,76.062000,103.500000,30.000000,78.000000,0.953000,18.000000,240.500000,84.545000,103.500000


In [58]:
df_test.describe()

,Salary,AvgPointsPerGame,atHome,injured,7GameAvg,FloorAvg,4GameAvg,LastGame,dvp,value,fvalue,odds,O/U,penalty,Final
count,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000
mean,4293.502825,17.824061,0.500000,0.132768,18.215352,11.903782,18.284165,17.728814,15.214336,25.761017,0.202556,0.081921,205.321328,-1033.993866,18.147982
std,1788.605342,11.572297,0.500353,0.339564,12.395035,9.960629,12.868369,15.081816,8.465373,10.731632,0.143131,7.495189,42.701368,2868.165839,14.324296
min,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.500000,1.000000,18.000000,0.000000,-12.500000,1.000000,-9149.199000,-0.500000
25%,3000.000000,9.262000,0.000000,0.000000,9.000000,4.107000,7.906500,5.250000,8.000000,18.000000,0.097500,-7.000000,208.000000,-8.196000,6.625000
50%,3500.000000,16.225500,0.500000,0.000000,16.411000,9.724000,16.343500,14.375000,15.500000,21.000000,0.193500,1.000000,214.500000,-1.660000,14.687500
75%,5100.000000,24.621250,1.000000,0.000000,25.152000,16.941750,25.375000,26.500000,22.000000,30.600000,0.290000,7.000000,222.000000,5.128000,26.188000
max,12800.000000,64.224000,1.000000,1.000000,68.607000,53.990000,70.062000,97.750000,30.000000,76.800000,0.953000,12.500000,230.500000,34.708000,70.750000


In [59]:
from sklearn.ensemble import RandomForestRegressor

modelRFR = RandomForestRegressor(n_estimators=20,
#                               random_state=1,
                              verbose=False,
                              warm_start=True)
modelRFR = modelRFR.fit(X_train, y_train)

yRF_prediction = modelRFR.predict(X_test)
print (modelRFR.score(  X_train , y_train ) , modelRFR.score(  X_test , y_test  ))
print ("\naccuracy", np.sum(abs(yRF_prediction-y_test)<5) / float(len(y_test)))
#51.8 at 120

0.950360512763 0.718630020802

accuracy 0.612994350282


In [60]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit( X_train , y_train )
print (svr_rbf.score(  X_train , y_train ) , svr_rbf.score(  X_test , y_test  ))
SVR_prediction = svr_rbf.predict(X_test)
print ("\naccuracy", np.sum(abs(SVR_prediction-y_test)<5) / float(len(y_test)))

0.785892168928 0.665305796093

accuracy 0.621468926554


In [61]:
#74 63
from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(max_depth=5)
regr.fit( X_train , y_train )
print (regr.score(  X_train , y_train ) , regr.score(  X_test , y_test  ))
DTR_prediction = regr.predict(X_test)
print ("\naccuracy", np.sum(abs(DTR_prediction-y_test)<5) / float(len(y_test)))

0.722667453853 0.742236435432

accuracy 0.631355932203


In [62]:
#70 63
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=7)
knr.fit( X_train , y_train )
print (knr.score(  X_train , y_train ) , knr.score(  X_test , y_test  ))
KNR_prediction = knr.predict(X_test)
print ("\naccuracy", np.sum(abs(KNR_prediction-y_test)<5) / float(len(y_test)))

0.755951850029 0.707819006879

accuracy 0.631355932203


In [63]:
#74 64
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor()
mlp.fit( X_train , y_train )
print (mlp.score(  X_train , y_train ) , mlp.score(  X_test , y_test  ))
mlp_prediction = mlp.predict(X_test)
print ("\naccuracy", np.sum(abs(mlp_prediction-y_test)<5) / float(len(y_test)))

0.715846397295 0.748561312107

accuracy 0.646892655367


In [64]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor\
    (activation='identity',solver='adam', learning_rate='adaptive', hidden_layer_sizes=(3, 6))
mlp.fit( X_train , y_train )
print (mlp.score(  X_train , y_train ) , mlp.score(  X_test , y_test  ))
mlp_prediction = mlp.predict(X_test)
print ("\naccuracy", np.sum(abs(mlp_prediction-y_test)<5) / float(len(y_test)))

0.706881955697 0.743078867602

accuracy 0.635593220339


In [65]:
# 0.3911
#y_tree_prediction
#pd.concat([dfTest["Name"],pd.DataFrame(y_tree_prediction),pd.DataFrame(y_test)],axis=1)

In [66]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(16, input_dim=len(features), init='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
model.compile(optimizer='adam', metrics=['accuracy','recall','fmeasure'],loss="mae")
# MAE 58.23%<5 16-8-1 50 Adam MAE
# MAE 55.69%<5 16-8-1  50 epoch SGD  MAE
# MSE 81% 
# MAPE 35% 
# MSLE 53% 
# HINGE 34% 
# SGD MSE 150 70% 
model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          verbose=False,
          nb_epoch=10, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))
model.evaluate(X_test,y_test)
y_predict = model.predict(X_test)
print ("\naccuracy", np.sum(abs(y_predict.T[0]-y_test)<5) / float(len(y_test)))
print(y_predict[:5])
#19% 3 layer


time taken 18.227442026138306 seconds
672/708 [===========================>..] - ETA: 0s
accuracy 0.666666666667
[[ 59.54344177]
 [ 56.0955658 ]
 [ 52.63440323]
 [ 45.96463776]
 [ 52.13122559]]


In [67]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(8, input_dim=len(features), init='normal', activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
model.compile(optimizer='adam', metrics=['accuracy','recall','fmeasure'],loss="mse")
# MAE 58.23%<5 16-8-1 50 Adam MAE
# MAE 55.69%<5 16-8-1  50 epoch SGD  MAE
# MSE 81% 
# MAPE 35% 
# MSLE 53% 
# HINGE 34% 
# SGD MSE 150 70% 
model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          verbose=False,
          nb_epoch=10, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))
model.evaluate(X_test,y_test)
y_predict = model.predict(X_test)
print ("\naccuracy", np.sum(abs(y_predict.T[0]-y_test)<5) / float(len(y_test)))
print(y_predict[:5])
#19% 3 layer


time taken 18.524656772613525 seconds
416/708 [================>.............] - ETA: 0s
accuracy 0.645480225989
[[ 57.10092163]
 [ 55.85467529]
 [ 52.62973785]
 [ 45.12490082]
 [ 51.44248199]]


In [162]:
kk = pd.DataFrame.from_dict(data= {"NN":y_predict.T[0] ,"RF":yRF_prediction})
kk.head()

,NN,RF
0,56.664860,53.4050
1,55.521008,47.2775
2,55.712742,53.5700
3,52.164722,51.9725
4,48.568645,50.8025


In [47]:
X_test

array([[ 3.86321586,  3.03397231,  3.05094403, ..., -0.07726129,
         1.06632353,  0.31396269],
       [ 3.63444739,  2.4766032 ,  3.03359608, ...,  0.42011201,
        -1.57242768,  1.68522672],
       [ 3.46287103,  2.76133685,  2.58190092, ...,  0.32800584,
        -0.14607568,  0.43320304],
       ..., 
       [-0.71215366, -0.79902664, -0.38565196, ...,  0.65958804,
        -0.28871088,  1.26788549],
       [-0.71215366, -0.25222215,  0.04577691, ...,  0.65958804,
         0.28182993,  0.13510217],
       [-0.71215366, -0.12357229, -0.04979893, ...,  0.65958804,
         0.28182993,  0.13510217]])

In [48]:
# df_test["NN"] = kk["NN"]
# df_test["RF"] = kk["RF"]
df_test[["NN","RF"]] = None
df_test[["NN", "RF"]] = kk
df_test.head()

KeyError: "['NN' 'RF'] not in index"

In [181]:
output = pd.concat([df_test,
            pd.DataFrame(y_predict.T[0],columns=["RF"]),
            pd.DataFrame(yRF_prediction,columns=["NN"])]
          ,axis=1)
#output.to_csv('data/output/2017-01-24.csv', sep=',', encoding='utf-8', index=False, float_format='%.3f')

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [171]:
output.head()

,0,1,2,3,4,5,6,7,8,9,10,11,RF,NN
0,3.443570,2.918456,2.922618,2.668608,3.323618,3.443570,-0.540922,-0.996791,2.315791,-0.072153,1.054632,0.311947,56.664860,53.4050
1,3.228400,2.330460,2.904395,2.471951,2.871254,3.228400,-1.236965,1.003219,2.559417,0.418265,-1.558122,1.685760,55.521008,47.2775
2,3.067022,2.630839,2.429893,2.678441,2.218369,3.067022,-0.516356,1.003219,1.654521,0.327447,-0.145823,0.431409,55.712742,53.5700
3,2.636681,2.629851,2.445136,2.811741,2.218369,2.636681,0.122365,1.003219,1.845941,0.509083,-1.628737,0.550871,52.164722,51.9725
4,2.582889,2.105400,2.283932,2.296075,2.470295,2.582889,-1.024058,-0.996791,1.393493,0.327447,0.136637,-1.241058,48.568645,50.8025


In [103]:
output.loc[:,("Name","NN","RF","AvgPointsPerGame","Final","O/U","odds","dvp")].sort_values(by=["O/U","NN","dvp"], ascending=False).head(10)

,Name,NN,RF,AvgPointsPerGame,Final,O/U,odds,dvp
164,James Michael McAdoo,NaN,NaN,4.850,13.50,230.5,0.0,29.00
11,Draymond Green,NaN,NaN,39.028,25.00,230.5,0.0,28.50
81,Zaza Pachulia,NaN,NaN,18.314,30.75,230.5,0.0,28.00
169,JaVale McGee,NaN,NaN,10.417,16.75,230.5,0.0,28.00
28,Mason Plumlee,NaN,NaN,31.021,36.75,230.5,8.5,22.00
1,Kevin Durant,NaN,NaN,49.851,55.00,230.5,0.0,18.50
72,Andre Iguodala,NaN,NaN,18.283,28.75,230.5,0.0,18.50
162,Meyers Leonard,NaN,NaN,11.970,4.50,230.5,8.5,16.00
9,Damian Lillard,NaN,NaN,43.227,35.00,230.5,8.5,15.50
15,C.J. McCollum,NaN,NaN,36.901,43.75,230.5,8.5,13.75


##  Predictions for Today!!!

In [21]:
scaler1 = StandardScaler()
features = ['Salary','AvgPointsPerGame','7GameAvg','FloorAvg'
            ,'4GameAvg','value','fvalue','atHome','LastGame','O/U','odds',"dvp"]

X_predict = scaler1.fit_transform(dfPredict[features].values)
y_tree_prediction = modelRFR.predict(X_predict) ## Random Forest
y_predict = model.predict(X_predict) ## Neural Networks prediction

In [22]:
output = pd.concat([dfPredict,
            pd.DataFrame(y_predict,columns=["NN"]),
            pd.DataFrame(y_tree_prediction,columns=["RF"])]
          ,axis=1)

In [551]:
#y_predict
#output.loc[:,"NN"].head()
#print(out_file)

In [23]:
output.to_csv(predict_file, sep=',', encoding='utf-8', index=False, float_format='%.3f')
withoutInjuries = output[(output["injured"]==0)]
withoutInjuries.head()
withoutInjuries.to_csv(out_file, sep=',', encoding='utf-8', index=False, float_format='%.3f')

In [207]:
from sklearn.metrics import mean_squared_error
rms = np.sqrt(mean_squared_error(y_test, y_predict))

In [208]:
print(rms)

7.08377222844


In [179]:
import pandas as pd 
import numpy as np
predicted_file = 'data/final/2017-01-28.csv'
print("Predict file " + predicted_file)
dfPredicted = pd.read_csv(predicted_file, header=0)
dfPredicted["ErrorNN"] = dfPredicted["NN"].T-dfPredicted["Final"]
dfPredicted["ErrorRF"] = dfPredicted["RF"].T-dfPredicted["Final"]
#rmsNN = np.sqrt(mean_squared_error(dfPredicted["Final"], dfPredicted["NN"]))
#rmsRF = np.sqrt(mean_squared_error(dfPredicted["Final"], dfPredicted["RF"]))
dfPredicted.iloc[:,-3:].describe()
print(np.sum(abs(dfPredicted["NN"].T-dfPredicted["Final"]) < 5)/float(len(dfPredicted["Final"])))
print(np.sum(abs(dfPredicted["RF"].T-dfPredicted["Final"]) < 5)/float(len(dfPredicted["Final"])))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorNN"]) < 5)]
print("Len NN > 5 " + str(len(dfPredictedFiltered)))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorNN"]) > 5)]
print("Len NN < 5 " + str(len(dfPredictedFiltered)))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorRF"]) < 5)]
print("Len RF > 5 " + str(len(dfPredictedFiltered)))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorRF"]) > 5)]
print("Len RF < 5 " + str(len(dfPredictedFiltered)))

dfPredictedFiltered.loc[:,("Name","NN","RF","Final","ErrorNN","ErrorRF")]

Predict file data/final/2017-01-28.csv
0.317307692308
0.341346153846
Len NN > 5 66
Len NN < 5 142
Len RF > 5 71
Len RF < 5 137


,Name,NN,RF,Final,ErrorNN,ErrorRF
0,DeMarcus Cousins,54.636,51.549,67.00,-12.364,-15.451
1,Karl-Anthony Towns,58.177,43.118,66.25,-8.073,-23.132
2,Giannis Antetokounmpo,45.161,45.064,39.50,5.661,5.564
3,Nikola Jokic,64.662,65.015,39.50,25.162,25.515
5,Isaiah Thomas,41.379,29.382,56.25,-14.871,-26.868
6,Eric Bledsoe,41.656,40.059,64.50,-22.844,-24.441
7,Stephen Curry,44.666,35.309,66.75,-22.084,-31.441
8,Chris Paul,13.443,10.995,66.75,-53.307,-55.755
10,Draymond Green,38.157,38.211,26.25,11.907,11.961
12,Marc Gasol,57.627,49.304,33.50,24.127,15.804
